In [1]:
# | default_exp wb_veg

In [2]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [3]:
# | export

import warnings
import collections
import numpy as np
from typing import Dict
from sureau_ecos_py.plant_utils import (
    plc_comp,
    rs_comp
)

from sureau_ecos_py.create_vegetation_parameters import (
    create_vegetation_parameters
)
from sureau_ecos_py.create_modeling_options import create_modeling_options
from sureau_ecos_py.create_soil_parameters import create_soil_parameters
from sureau_ecos_py.create_stand_parameters import create_stand_parameters

In [4]:
# | export
def new_wb_veg(veg_params:Dict # Dictionary created using the `create_vegetation_parameters` function
) -> Dict:

    "Create an object wb_veg from veg_params"

    # Assert parameters ---------------------------------------------------------
    assert (
        isinstance(veg_params, Dict)
    ), f"veg_param must be a Dictionary not a {type(veg_params)}"


    # Add params to wb_veg dictionary -------------------------------------------

    # Create Empty dict
    wb_veg = collections.defaultdict(list)

    # Add veg_params
    wb_veg["params"] = veg_params

    # Add Stem and Leaf turgor point
    wb_veg['params']['psi_tlp_leaf'] = wb_veg["params"]["pifullturgor_leaf"]*wb_veg["params"]["epsilonsym_leaf"]/(wb_veg["params"]["pifullturgor_leaf"]+wb_veg["params"]["epsilonsym_leaf"])
    wb_veg['params']['psi_tlp_stem'] = wb_veg["params"]["pifullturgor_stem"]*wb_veg["params"]["epsilonsym_stem"]/(wb_veg["params"]["pifullturgor_stem"]+wb_veg["params"]["epsilonsym_stem"])

    # Add potentials
    wb_veg['psi_lapo'] = 0
    wb_veg['psi_sapo'] = 0
    wb_veg['psi_lsym'] = 0
    wb_veg['psi_ssym'] = 0

    # FP replaced "mem" by "cav" (when cavitation starts)
    wb_veg['psi_lapo_cav'] = 0
    wb_veg['psi_sapo_cav'] = 0
    wb_veg['psi_all_soil'] = 0

    # Conductance & capacitance (mmol/m2/s/MPa) Here on leaf area basis but they
    # are to be updated as a function symplasm conductance and leaf area hydraulic
    # conductances

    # constant value during simulation
    wb_veg["k_plant"] =  wb_veg["params"]["k_plant_init"]

    # constant value during simulation
    wb_veg["k_lsym"]  =  wb_veg["params"]["k_lsym_init"]

    # constant value during simulation
    wb_veg["k_ssym"]  =  wb_veg["params"]["k_ssyminit"]

    # Value is updated in compute_kplant_wb_veg
    wb_veg["k_rsapo"] =  np.nan

    # Value is updated in compute_kplant_wb_veg
    wb_veg["k_slapo"] =  np.nan

    # Conductance rhisophere for each soil layer
    wb_veg['k_soil_to_stem'] = np.array([np.nan, np.nan, np.nan], dtype=float)

    # Capacitances
    # Symplasm capacitance updated as a function of water content and Leaf area
    # (mmol/m2leaf/MPa) /updated in update.capacitancesSymAndApo()
    # (NM : 25/10/2021)

    wb_veg['c_lsym'] = np.nan
    wb_veg['c_sapo'] = np.nan

    # Leaf and canopy conductance

    # initialised at 0 to compute Tleaf on first time step considering gs = 0
    # and not NA s
    wb_veg['gmin'] = 0

    # Gmin for stem and branches
    wb_veg['gmin_s'] = wb_veg["params"]["gmin_s"]

    # TODO voir si y'a besoin d'initialiser
    # see if there is a need to initialize

    wb_veg['regul_fact'] = 0.01

    # TODO voir pour mettre tout en NA si TranspirationMod = 0
    # see to put everything in NA if TranspirationMod = 0

    wb_veg['gs_bound'] = np.nan

    # initialised to 0 to compute Tleaf on first time step considering gs = 0
    # and not NA
    wb_veg['gs_lim'] = 0
    wb_veg['gcanopy_bound'] = np.nan
    wb_veg['gcanopy_lim'] = np.nan
    wb_veg['g_bl'] = np.nan
    wb_veg['g_crown'] = np.nan

    # Fluxes
    wb_veg['e_prime'] = 0
    wb_veg['e_min'] = 0
    wb_veg['e_min_s'] = 0
    wb_veg['e_bound'] = 0
    wb_veg['e_lim'] = 0
    wb_veg['flux_soil_to_stem'] = np.zeros(3)
    wb_veg['transpiration_mm'] = 0
    wb_veg['e_min_mm'] = 0
    wb_veg['e_min_s_mm'] = 0

    # LAI and LAI-dependent variables
    wb_veg['lai_pheno'] = np.zeros(1)
    wb_veg['lai']      = np.zeros(1)
    wb_veg['canopy_storage_capacity'] = np.zeros(1)

    # rainfall and interception

    # precipitation (ppt) that reach the soil
    wb_veg['ppt_soil'] = 0

    # interceptedWater /quantite d'eau dans la canopee
    wb_veg['intercepted_water_amount'] = 0
    wb_veg['evaporation_intercepted'] = 0
    wb_veg['etpr'] = 0

    # defoliation // no defoliation (add an option to set defoliation due to
    # cavitation of the Plant Above)
    wb_veg['defoliation'] = 0
    wb_veg['lai_dead'] = 0

    # Cavitation
    # percent loss of conductivity in the leaf [%]
    wb_veg["plc_leaf"] = 0

    # percent loss of conductivity in the stem [%]
    wb_veg["plc_stem"] = 0

    # leaf temp
    wb_veg["leaf_temperature"] = np.nan

    # Pheno, parameters if deciduous
    if wb_veg["params"]["foliage"] == "deciduous":

        wb_veg['lai_pheno'] = 0

        # temmpeerature sum to determine budburst
        wb_veg['sum_temperature'] = 0

        # budburst date
        wb_veg['bud_burst_date'] = np.nan

        print("wb_veg params for deciduous forest created")
    else:
        print(f'wb_veg params for {wb_veg["params"]["foliage"]} created')

    # water storage in canopy

    # Fuel moisture content of the dead compartment (gH20/gMS)
    wb_veg['dfmc']     = np.nan

    # Live fuel moisture content of the apoplasmic compartment (gH20/gMS)
    wb_veg['lfmc_apo']  = np.nan

    # Live fuel moisture content of the apoplasmic compartment (gH20/gMS)
    wb_veg['lfmc_symp'] = np.nan

    # live fuel moisture content (gH20/gMS)
    wb_veg['lfmc']     = np.nan

    # Live Canopy dry matter [gMS/m2 soil]
    wb_veg['dm_live_canopy']  = np.nan

    # Dead Canopy dry matter [gMS/m2 soil]
    wb_veg['dm_dead_canopy']  = 0

    # Q leaf apo (mol/m2leaf)
    wb_veg['q_lapo_sat_mmol'] = 0
    wb_veg['q_lapo_sat_l'] = 0

    # Q stem apo (mol/m2leaf)
    wb_veg['q_sapo_sat_mmol'] = 0
    wb_veg['q_sapo_sat_l'] = 0

    # Q leaf symplasm (mol/m2leaf)
    wb_veg['q_lsym_sat_mmol'] = 0
    wb_veg['q_lsym_sat_l']    = 0

    # Q Stem symplasm (mol/m2leaf)
    wb_veg['q_ssym_sat_mmol'] = 0
    wb_veg['q_ssym_sat_l']    = 0

    # Q Stem and Leaf apo and symp in liter/kg TODO 13/08/2021: better in mmol?
    wb_veg['q_lapo_l'] = 0
    wb_veg['q_sapo_l'] = 0
    wb_veg['q_lsym_l'] = 0
    wb_veg['q_ssym_l'] = 0

    wb_veg['delta_q_lapo_mmol_diag'] = 0

    # Water from the cavitated part, term from equations 6 and 7 in the
    # SurEau-Ecos paper
    wb_veg['f_l_cav'] = 0
    wb_veg['f_s_cav'] = 0

    # Compute PLC

    # Leaf
    wb_veg['plc_leaf'] = plc_comp(psi = wb_veg['psi_lapo'],
                                  slope = wb_veg['params']['slope_vc_leaf'],
                                  p50 = wb_veg['params']['p50_vc_leaf']
                                  )
    # Stem
    wb_veg['plc_stem'] = plc_comp(psi = wb_veg['psi_sapo'],
                                  slope = wb_veg['params']['slope_vc_stem'],
                                  p50 = wb_veg['params']['p50_vc_stem']
                                  )


    return wb_veg


#### __Example: Create wb_veg dictionary__

In [5]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="sigmoid",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="campbell",
)

In [6]:
stand_params = create_stand_parameters(
    file_path="./sample_data/stand_example.csv",
    lai_max=None,
    latitude=None,
    longitude=None,
    sep=";",
)

In [7]:
soil_params = create_soil_parameters(
    file_path="./sample_data/soil_example.csv",
    modeling_options=modeling_options_dict,
    default_soil=False,
    offset_psoil=1,
    psoil_at_field_capacity=-1,
)

There is an offset on Psoil of 1 MPa
Psoil at field capacity = -0.001 MPa
You are using campbell pedotransfer formulation
Available water capacity Wilting: 273.74477742718403 mm
Available water capacity Residual: 307.81195119276424 mm
Can soil_params["v_soil_storage_capacity"] be negative?? Ask


In [8]:
vegetation_params = create_vegetation_parameters(
    file_path="./sample_data/vegetation_example_deciduous_beech_wide.csv",
    list_of_parameters=None,
    soil_parameters=soil_params,
    stand_parameters=stand_params,
    modeling_options=modeling_options_dict,
    sep=",",
)

frac_leaf_sym' set to 0.4
Available water capacity @Tlp (Campbell):278.27401951129264 mm
Available water capacity @P50 (Campbell):282.61737155892024 mm


/workspaces/sureau_ecos_py/sureau_ecos_py/create_vegetation_parameters.py:106: UserWarning: Make sure that depth from soil_parameters is in meters
  warnings.warn("Make sure that depth from soil_parameters is in meters")


In [9]:
new_wb_veg(vegetation_params)

wb_veg params for deciduous forest created


defaultdict(list,
            {'params': defaultdict(list,
                         {'foliage': 'deciduous',
                          'c_lapoinit': 1e-05,
                          'c_sapoinit': 2e-05,
                          'f_crit': 500,
                          'jarvis_par': 0.006,
                          'k': 0.5,
                          'ldmc': 514,
                          'lma': 91,
                          'p12_gs': -1.26,
                          'p50_vc_leaf': -3.15,
                          'p50_vc_stem': -3.15,
                          'p88_gs': -2,
                          'pt_coeff': 1.14,
                          'pifullturgor_leaf': -1.8,
                          'pifullturgor_stem': -1.8,
                          'psi_close': -2.4,
                          'psistartclosing': -1.26,
                          'q10_1_gmin': 1.2,
                          'q10_2_gmin': 2.1,
                          'tphase_gmin': 40,
                          't_base': 

In [10]:
# | export
def compute_pheno_wb_veg(wb_veg:Dict, # Dictionary created using the `new_wb_veg` function
                         temperature:float, # daily tempeature  (degC)
                         day_of_year:int,
                         ) -> Dict:

    "Compute phenology and leaf area index (lai) for wb_veg dictionary in sureau_ecos_py"
    # Assert parameters ---------------------------------------------------------

    # wb_veg
    assert (
        isinstance(wb_veg, Dict)
    ), f"wb_veg must be a Dictionary not a {type(wb_veg)}"

    # Temperature
    assert (
        -40 <= temperature <= 70
    ), "Unrealistic air temperature, value must be a value between -40 and 70"

    # Day of year
    assert (
        isinstance(day_of_year, int) and 366 >= day_of_year >= 1
    ), "day_of_year must be a integer value between 1-366"


    # Set to initial parameters at the beggining of the year --------------------
    if day_of_year == 1:
        print(f'Setting initial parameters at the beggining of the year (day number:{day_of_year})')

        if wb_veg['params']['foliage'] ==  "evergreen":

            wb_veg['lai_pheno'] = wb_veg['params']['lai_max']

        elif wb_veg['params']['foliage'] ==  "deciduous":
            wb_veg['lai_pheno'] = 0
            wb_veg['sum_temperature'] = 0
            wb_veg['bud_burst_date'] = np.nan

        elif wb_veg['params']['foliage'] ==  "forced":
            wb_veg['lai_pheno'] = 0
            wb_veg['sum_temperature'] = 0
            wb_veg['bud_burst_date'] = np.nan

        else:
            raise ValueError(
                'Error in setting initial parameters in compute_pheno_wb_veg function'
        )

    # Setting parameters for deciduous vegetation -------------------------------
    if wb_veg['params']['foliage'] ==  "deciduous":

        # if no budburst
        if np.isnan(wb_veg['bud_burst_date']):

            if temperature > wb_veg['params']['t_base'] and day_of_year >= wb_veg['params']['day_start']:

                # Update sum_temperature if temp > t_base
                wb_veg['sum_temperature'] = wb_veg['sum_temperature'] + temperature

            elif wb_veg['sum_temperature'] > wb_veg['params']['f_crit']:
                wb_veg['bud_burst_date'] = day_of_year

            else:
                raise ValueError(
                'Error setting sum_temperature for deciduous vegetation in compute_pheno_wb_veg function'
                )

        # loss of leaves on day 270
        elif day_of_year >= 280:
            wb_veg['lai_pheno'] = np.maximum(0, wb_veg['lai'] - np.maximum(0, wb_veg['params']['lai_max']/wb_veg['params']['nbday_lai']))


        elif np.isnan(wb_veg['bud_burst_date']) is False:

            # if bud break and leaf construction period
            if day_of_year < wb_veg['bud_burst_date'] + wb_veg['params']['nbday_lai']:
                wb_veg['lai_pheno'] = wb_veg['lai_pheno'] + np.maximum(0, wb_veg['params']['lai_max']/wb_veg['params']['nbday_lai'])

            else:
                raise ValueError(
                    'Error setting lai_pheno for deciduous vegetation in compute_pheno_wb_veg function'
                )

        else:
            raise ValueError(
                'Error setting parameters for deciduous vegetation in compute_pheno_wb_veg function'
                )

    # Setting parameters for forced vegetation ----------------------------------
    if wb_veg['params']['foliage'] ==  "forced":

        # No bud_burst_date
        if np.isnan(wb_veg['bud_burst_date']):
            if day_of_year >= wb_veg['params']['day_start_forced']:
                wb_veg['bud_burst_date'] = day_of_year

            else:
                raise ValueError(
                    'Error setting bud_burst_date for forced vegetation in compute_pheno_wb_veg function'
                )

        #  Leaf shedding
        elif day_of_year >= wb_veg['params']['day_end_forced']:
            wb_veg['lai_pheno'] = np.maximum(0, wb_veg['lai'] - np.maximum(0, wb_veg['params']['lai_max']/wb_veg['params']['nbday_lai']))

        elif np.isnan(wb_veg['bud_burst_date']) is False:

            # After bud burst and before full LAI
            if day_of_year < wb_veg['bud_burst_date'] + wb_veg['params']['nbday_lai']:
                wb_veg['lai_pheno'] = wb_veg['lai_pheno'] + np.maximum(0, wb_veg['params']['lai_max']/wb_veg['params']['nbday_lai'])

            else:
                raise ValueError(
                    'Error setting lai_pheno for forced vegetation in compute_pheno_wb_veg function'
                    )

        else:
            raise ValueError(
                'Error setting parameters for forced vegetation in compute_pheno_wb_veg function'
                )


    return wb_veg

#### __Example: Compute phenology for wb_veg dictionary__

In [11]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="sigmoid",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="campbell",
)

In [12]:
stand_params = create_stand_parameters(
    file_path="./sample_data/stand_example.csv",
    lai_max=None,
    latitude=None,
    longitude=None,
    sep=";",
)

In [13]:
soil_params = create_soil_parameters(
    file_path="./sample_data/soil_example.csv",
    modeling_options=modeling_options_dict,
    default_soil=False,
    offset_psoil=1,
    psoil_at_field_capacity=-1,
)

There is an offset on Psoil of 1 MPa
Psoil at field capacity = -0.001 MPa
You are using campbell pedotransfer formulation
Available water capacity Wilting: 273.74477742718403 mm
Available water capacity Residual: 307.81195119276424 mm
Can soil_params["v_soil_storage_capacity"] be negative?? Ask


In [14]:
vegetation_params_evergreen = create_vegetation_parameters(
    file_path="./sample_data/vegetation_example_wide.csv",
    list_of_parameters=None,
    soil_parameters=soil_params,
    stand_parameters=stand_params,
    modeling_options=modeling_options_dict,
    sep=",",
)

frac_leaf_sym' set to 0.4
Available water capacity @Tlp (Campbell):280.7301527422195 mm
Available water capacity @P50 (Campbell):283.4407203700769 mm


/workspaces/sureau_ecos_py/sureau_ecos_py/plant_utils.py:902: UserWarning: Foliage evergreen has no params
  warnings.warn("Foliage evergreen has no params")
/workspaces/sureau_ecos_py/sureau_ecos_py/create_vegetation_parameters.py:106: UserWarning: Make sure that depth from soil_parameters is in meters
  warnings.warn("Make sure that depth from soil_parameters is in meters")


In [15]:
wb_veg = new_wb_veg(vegetation_params)

wb_veg params for deciduous forest created


In [16]:
compute_pheno_wb_veg(wb_veg=wb_veg,
                     temperature=20,
                     day_of_year=1
)

Setting initial parameters at the beggining of the year (day number:1)


defaultdict(list,
            {'params': defaultdict(list,
                         {'foliage': 'deciduous',
                          'c_lapoinit': 1e-05,
                          'c_sapoinit': 2e-05,
                          'f_crit': 500,
                          'jarvis_par': 0.006,
                          'k': 0.5,
                          'ldmc': 514,
                          'lma': 91,
                          'p12_gs': -1.26,
                          'p50_vc_leaf': -3.15,
                          'p50_vc_stem': -3.15,
                          'p88_gs': -2,
                          'pt_coeff': 1.14,
                          'pifullturgor_leaf': -1.8,
                          'pifullturgor_stem': -1.8,
                          'psi_close': -2.4,
                          'psistartclosing': -1.26,
                          'q10_1_gmin': 1.2,
                          'q10_2_gmin': 2.1,
                          'tphase_gmin': 40,
                          't_base': 

In [ ]:
# | export
def update_lai_and_stocks_wb_veg(wb_veg:Dict, # Dictionary created using the `new_wb_veg` function
                                modeling_options: Dict,  # Dictionary created using the `create_modeling_options` function
                                ) -> Dict:

    # Assert parameters ---------------------------------------------------------

    # wb_veg
    assert (
        isinstance(wb_veg, Dict)
    ), f"wb_veg must be a Dictionary not a {type(wb_veg)}"


    # Make sure that modeling_options is a dictionary
    assert isinstance(
        modeling_options, Dict
    ), f"modeling_options must be a dictionary not a {type(modeling_options)}"

    #
    pass





In [32]:
# | export
def update_capacitances_apo_and_sym_wb_veg(wb_veg:Dict, # Dictionary created using the `new_wb_veg` function
    ) -> Dict:

    "Update symplasmic plant capacitances for trunk and leaves"

    # Assert parameters ---------------------------------------------------------
    # wb_veg
    assert (
        isinstance(wb_veg, Dict)
    ), f"wb_veg must be a Dictionary not a {type(wb_veg)}"

    # NM minimal double to avoid-INF
    dbxmin = 1e-100

    # Compute the relative water content of the symplasm ------------------------
    rwc_lsym = 1 - rs_comp(pi_ft = wb_veg['params']['pifullturgor_leaf'],
                           e_symp = wb_veg['params']['epsilonsym_leaf'],
                           psi = wb_veg['psi_lsym'] - dbxmin
                           )
    # Compute the derivative of the relative water content of the symplasm
    if wb_veg['psi_lsym'] > wb_veg['params']['psi_tlp_leaf']:

        # FP derivative of -Pi0- Eps(1-RWC)+Pi0/RWC
        rwc_lsym_prime = rwc_lsym / (-wb_veg['params']['pifullturgor_leaf'] - wb_veg['psi_lsym'] - wb_veg['params']['epsilonsym_leaf'] + 2 * wb_veg['params']['epsilonsym_leaf'] * rwc_lsym)

    else:
        # FP derivative of Pi0/Psi
        rwc_lsym_prime = -wb_veg['params']['pifullturgor_leaf'] / wb_veg['psi_lsym']**2


    # Compute the leaf capacitance (mmol/MPa/m2_sol)
    if wb_veg['lai'] == 0:
        wb_veg['c_lsym'] = 0

    else:
        wb_veg['c_lsym'] = wb_veg['q_lsym_sat_mmol_per_leaf_area'] * rwc_lsym_prime

    # Stem symplasmic canopy water content --------------------------------------
    rwc_ssym = 1 - rs_comp(pi_ft = wb_veg['params']['pifullturgor_stem'],
                           e_symp = wb_veg['params']['epsilonsym_stem'],
                           psi = wb_veg['psi_ssym'] - dbxmin
                           )


    # Compute the derivative of the relative water content of the symplasm
    if wb_veg['psi_ssym'] > wb_veg['params']['psi_tlp_stem']:

        # FP derivative of -Pi0- Eps(1-RWC)+Pi0/RWC
        rwc_ssym_prime = rwc_ssym / (-wb_veg['params']['pifullturgor_stem'] - wb_veg['psi_ssym'] - wb_veg['params']['epsilonsym_stem'] + 2 * wb_veg['params']['epsilonsym_stem'] * rwc_ssym)

    else:
        # FP derivative of Pi0/Psi
        rwc_ssym_prime = -wb_veg['params']['pifullturgor_stem'] / wb_veg['psi_ssym']**2


    # Compute the capacitance (mmol/MPa/m2_leaf)
    # Stem capacitance per leaf area can only decrease with LAI
    # (cannot increase when LAI<1 )
    wb_veg['c_ssym'] = wb_veg['q_ssym_sat_mmol_per_leaf_area'] * rwc_ssym_prime

    # Add c_sapo and c_lapo -----------------------------------------------------
    wb_veg['c_sapo'] = wb_veg['params']['c_sapoinit']
    wb_veg['c_lapo'] = wb_veg['params']['c_lapoinit']

    return wb_veg


#### __Example: __

In [31]:
update_capacitances_apo_and_sym_wb_veg(wb_veg)

defaultdict(list,
            {'params': defaultdict(list,
                         {'foliage': 'deciduous',
                          'c_lapoinit': 1e-05,
                          'c_sapoinit': 2e-05,
                          'f_crit': 500,
                          'jarvis_par': 0.006,
                          'k': 0.5,
                          'ldmc': 514,
                          'lma': 91,
                          'p12_gs': -1.26,
                          'p50_vc_leaf': -3.15,
                          'p50_vc_stem': -3.15,
                          'p88_gs': -2,
                          'pt_coeff': 1.14,
                          'pifullturgor_leaf': -1.8,
                          'pifullturgor_stem': -1.8,
                          'psi_close': -2.4,
                          'psistartclosing': -1.26,
                          'q10_1_gmin': 1.2,
                          'q10_2_gmin': 2.1,
                          'tphase_gmin': 40,
                          't_base': 

#### __Example: __